In [2]:
import codecs
#finds all pathnames matching a pattern, like regex
import glob
import logging
import multiprocessing
import os
import pprint
#regular expressions
import re
import nltk
#word 2 vec
import gensim.models.word2vec as w2v
#dimensionality reduction
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#visualization
import seaborn as sns


ModuleNotFoundError: No module named 'gensim'

In [ ]:
%pylab inline

In [3]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to C:\Users\Sanjay
[nltk_data]     Mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sanjay
[nltk_data]     Mishra\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
#step 1 process data => convert to tokens 
#Prepare corpus
book_filenames = sorted(glob.glob("data/*.txt"))
print("Found books:")
book_filenames

Found books:


['data\\got1.txt',
 'data\\got2.txt',
 'data\\got3.txt',
 'data\\got4.txt',
 'data\\got5.txt']

In [5]:
#combine books in to one string
corpus_raw = u""
#initialize rawunicode , we'll add all text to this one  file in memory
#for each book, read it, open it un utf 8 format, 
#add it to the raw corpus
for book_filename in book_filenames:
    print("Reading '{0}'...".format(book_filename))
    with codecs.open(book_filename, "r", "utf-8") as book_file:
        corpus_raw += book_file.read()
print("Corpus is now {0} characters long".format(len(corpus_raw)))




Reading 'data\got1.txt'...
Reading 'data\got2.txt'...
Reading 'data\got3.txt'...
Reading 'data\got4.txt'...
Reading 'data\got5.txt'...
Corpus is now 9719485 characters long


In [6]:
#tokenizastion! saved the trained model here
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [7]:
#tokenize into sentences
raw_sentences = tokenizer.tokenize(corpus_raw)

In [8]:
#convert into list of words
#remove unecessary characters, split into words, no hyhens and sh**
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words



In [9]:
#for each sentence, sentences where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))


In [10]:
print(raw_sentences[5])

Heraldic crest by Virginia Norey.


In [11]:
print(sentence_to_wordlist(raw_sentences[5]))

['Heraldic', 'crest', 'by', 'Virginia', 'Norey']


In [12]:
#count tokens, each one being a sentence
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 1,818,103 tokens


In [15]:
#step 2 build our model
# Train word2Vector
#ONCE we have vectors
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING
 #Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized

num_features = 300
# Minimum word count threshold.
min_word_count = 3
# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()
print(num_workers)

# Context window length.
context_size = 7
# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3
# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1


4


In [16]:
thrones2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

NameError: name 'w2v' is not defined

In [17]:
thrones2vec.build_vocab(sentences)

NameError: name 'thrones2vec' is not defined

In [18]:
print("Word2Vec vocabulary length:", len(thrones2vec.vocab))

NameError: name 'thrones2vec' is not defined

In [19]:
#train model on sentneces
thrones2vec.train(sentences)

NameError: name 'thrones2vec' is not defined

In [20]:
#save model
if not os.path.exists("trained"):
    os.makedirs("trained")

In [21]:
thrones2vec.save(os.path.join("trained", "thrones2vec.w2v"))

NameError: name 'thrones2vec' is not defined

In [22]:
#load model
thrones2vec = w2v.Word2Vec.load(os.path.join("trained", "thrones2vec.w2v"))

NameError: name 'w2v' is not defined

In [23]:
#compress the model in to 2D space and plot them
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)
#put it all into a giant matrix
all_word_vectors_matrix = thrones2vec.syn0
all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix)

NameError: name 'sklearn' is not defined

In [24]:
#plot point in 2d space
points = pd.DataFrame(
    [
        (word, coords[0], coords[1])
        for word, coords in [
            (word, all_word_vectors_matrix_2d[thrones2vec.vocab[word].index])
            for word in thrones2vec.vocab
        ]
    ],
    columns=["word", "x", "y"]
)

NameError: name 'pd' is not defined

In [25]:
points.head(10)

NameError: name 'points' is not defined

In [26]:
#plot
sns.set_context("poster")

NameError: name 'sns' is not defined

In [27]:
points.plot.scatter("x", "y", s=10, figsize=(20, 12))

NameError: name 'points' is not defined

In [28]:
#Zoom in to some interesting places
def plot_region(x_bounds, y_bounds):
    slice = points[
        (x_bounds[0] <= points.x) &
        (points.x <= x_bounds[1]) & 
        (y_bounds[0] <= points.y) &
        (points.y <= y_bounds[1])
    ]
    
    ax = slice.plot.scatter("x", "y", s=35, figsize=(10, 8))
    for i, point in slice.iterrows():
        ax.text(point.x + 0.005, point.y + 0.005, point.word, fontsize=11)

In [30]:
plot_region(x_bounds=(4.0, 4.2), y_bounds=(-0.5, -0.1))
plot_region(x_bounds=(0, 1), y_bounds=(4, 4.5))


NameError: name 'points' is not defined

In [31]:
#Words closest to the given word
thrones2vec.most_similar("Stark")

NameError: name 'thrones2vec' is not defined

In [32]:
thrones2vec.most_similar("Aerys")


NameError: name 'thrones2vec' is not defined

In [33]:
thrones2vec.most_similar("direwolf")

NameError: name 'thrones2vec' is not defined

In [34]:
#Linear relationships between word pairs
#distance, similarity, and ranking
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = thrones2vec.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{start1} is related to {end1}, as {start2} is related to {end2}".format(**locals()))
    return start2

In [35]:
nearest_similarity_cosmul("Stark", "Winterfell", "Riverrun")
nearest_similarity_cosmul("Jaime", "sword", "wine")
nearest_similarity_cosmul("Arya", "Nymeria", "dragons")

NameError: name 'thrones2vec' is not defined